# Evaluating performance of scalable GPLVM models for dimensionality reduction of single-cell genomics datasets
---

In [ ]:
## Load output of trained GPLVM

## Do we require non-random initialization of the GPLVM model?

In the GASPACHO paper we used PCA as the inital state, but is this really necessary?

### 1. GPLVM initialized from the PCA doesn't learn anything different

In [1]:
## ... correlation between PCs and LVs with and without initialization

### 2. Does GPLVM trained with random initialization capture cell type identity?

As a first impression we can visualize UMAP embeddings from all the dimensionality reductions (random GPLVM, initialized GPLM, PCA) and assess qualitatively if we see a separation between cells of the same cell type

In [2]:
## code for UMAP plots ##

We can quantify the agreement between clusters obtained with distance on the LVs with ground truth cell type labels from a range of datasets, using metrics for [clustering evaluation](https://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation) implemented in `sklearn` (e.g. Adjusted Rand Index and Normalized Mutual Information). We then compare the ARI and NMI values we get from GP latent variables with those that we get from clustering on PCs.

In [ ]:
## dummy code just to have an idea, I have not run this

# Cluster cells based on GPLVM dimensions (with random initialization)
sc.pp.neighbors(adata, use_rep="X_GPLVM_random", key_added='gplvm_random')
sc.tl.leiden(adata, neighbors_key='gplvm_random', key_added='clusters_GPLVM_random')

# Cluster cells based on GPLVM dimensions (with PCA initialization)
sc.pp.neighbors(adata, use_rep="X_GPLVM_init", key_added='gplvm_init')
sc.tl.leiden(adata, neighbors_key='gplvm_init', key_added='clusters_GPLVM_init')

# Cluster cells based on PC dimensions
sc.pp.neighbors(adata, use_rep="X_pca", key_added='pca')
sc.tl.leiden(adata, neighbors_key='pca', key_added='clusters_PCA')

# Compute agreement metrics
## Adjusted Rand Index
ARI_gplvm_rand = metrics.adjusted_rand_score(adata.obs['celltype'], adata.obs['clusters_GPLVM_random'])
ARI_gplvm_init = metrics.adjusted_rand_score(adata.obs['celltype'], adata.obs['clusters_GPLVM_random'])
ARI_pca = metrics.adjusted_rand_score(adata.obs['celltype'], adata.obs['clusters_PCA'])
## Normalized Mutual Information
# ...

Here `ARI_pca` and `ARI_gplvm_init` should be rather similar, but are they higher than `ARI_gplvm_rand`? It might also make sense to compute each metric for each celltype cluster, to have a distribution of values rather than a single score

In [ ]:
## summarise results from multiple datasets in one plot

At the end of this analysis we should be able to tell whether the random initialization of GPLVM is as good as PCA as a dimensionality reduction strategy for scRNA-seq data, or whether initialization is needed to capture cell type identity. 

## Part 2: incorporating a latent variable with periodic prior in the GPLVM model allows to correct cell cycle effects

Having shown that GPLVMs (either with or without initialization) can perform the same tasks that PCA performs on scRNA-seq datasets, we now turn to evaluating the advantages of this model. One of the main advantages is that we can assign informative priors on some of the latent variables to better disentangle nuisance effects in gene expression data. One such effect is the cell cycle: by incorporating a latent variable with periodic kernel, we force that variable to capture the main "cyclic" effect in the dataset.

### 1. Does the variable with periodic kernel really capture the cell cycle?
Remember the model has no idea what the cell cycle is or which genes are related to it

In [3]:
## Plot scatter plots or heatmap of cell cycle pseudotime VS expression of cell cycle genes 

### 2. When we exclude the latent variable capturing the cell cycle, does this reduce separation between cells in different stages in clustering?

For this analysis we start by using the pancreas dataset in the scvelo package, where we observed that the clustering and embedding based on PCA still retains separation between cells driven by proliferation (are there more datasets like this?).

We want to evaluate whether excluding the cell cycle latent variable creates a reduced dimensionality space where cells with the same label but different proliferation status are well mixed, while still maintaining differences. We need to define a metric to quantify mixing (examples metrics to quantify batch mixing [here](https://github.com/theislab/scib#metrics)). 

In [ ]:
## Code to identify proliferating cells

In [ ]:
## Code to quantify mixing between proliferating cells of the same type

As an additional positive control, we can check how much the mixing is reduced if we do include the cell cycle latent variable